In [5]:
import pyrealsense2 as rs
import cv2
import numpy as np


# Set up a mutex to share data between threads 设置互斥体以在线程之间共享数据
from threading import Lock
frame_mutex = Lock()
frame_data = {"left"  : None,
              "right" : None,
              "timestamp_ms" : None
              }

def callback(frame):
    global frame_data
    if frame.is_frameset():
        frameset = frame.as_frameset()
        f1 = frameset.get_fisheye_frame(1).as_video_frame()
        f2 = frameset.get_fisheye_frame(2).as_video_frame()
        left_data = np.asanyarray(f1.get_data())
        right_data = np.asanyarray(f2.get_data())
        ts = frameset.get_timestamp()
        frame_mutex.acquire()
        frame_data["left"] = left_data
        frame_data["right"] = right_data
        frame_data["timestamp_ms"] = ts
        frame_mutex.release()
pipe = rs.pipeline()

# Build config object and request pose data
cfg = rs.config()
cfg.enable_stream(rs.stream.pose)

# Start streaming with requested config

pipe.start(cfg, callback)
camera = cv2.VideoCapture(1)
while True:
    # reg, img = camera.read()
    # cv2.imshow('Camera', img)
    # Wait for the next set of frames from the camera
    frames = pipe.wait_for_frames()
    f1=frames.get_fisheye_frame(1).as_video_frame()
    img = np.asanyarray((f1).get_data())
    print(img)
    cv2.imshow(img)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

pipe.stop() 

RuntimeError: null pointer passed for argument "frame_ref"